In [ ]:
from depmapomics.config import *
from depmap_omics_upload import tracker
from depmap_omics_upload import upload
from depmapomics import mutations
from genepy.utils import helper as h

In [ ]:
wgs_mat = mutations.generateGermlineMatrix(refworkspace=WGSWORKSPACE,
        vcfdir=WGSVCFDIR,
        savedir="output/" + SAMPLESETNAME + "/",
        filename="binary_mutguides_wgs.tsv.gz",
        bed_location=GUIDESBED,)

In [ ]:
from depmap_omics_upload import tracker, upload
mytracker = tracker.SampleTracker()

In [ ]:
upload.initVirtualDatasets(samplesetname="22Q2dryrun")

In [ ]:
from depmap_omics_upload import tracker, upload

virtual_dryrun = {'internal': 'a3b993e833694883bd242b5316f9c9a0',
 'dmc': 'af9f25e12bde4ade8558bbe07caeab5a',
 'public': '1368629a38d84323b50151512d575b8b'}

In [ ]:
from depmapomics.config import *

TAIGA_CN = "cn-latest-8bea"
TAIGA_EXPRESSION = "expression-869e"

LATEST2FN_NUMMAT_MODEL = {
#     TAIGA_CN: VIRTUAL_FILENAMES_NUMMAT_CN_MODEL,
#     TAIGA_EXPRESSION: VIRTUAL_FILENAMES_NUMMAT_EXP_MODEL,
     TAIGA_MUTATION: VIRTUAL_FILENAMES_NUMMAT_MUT_MODEL,
}

LATEST2FN_TABLE_MODEL = {
#    TAIGA_CN: VIRTUAL_FILENAMES_TABLE_CN_MODEL,
#     TAIGA_FUSION: VIRTUAL_FILENAMES_TABLE_FUSION_MODEL,
     TAIGA_MUTATION: VIRTUAL_FILENAMES_TABLE_MUT_MODEL,
}

LATEST2FN_NUMMAT_PR = {
#   TAIGA_CN: VIRTUAL_FILENAMES_NUMMAT_CN_PR,
#     TAIGA_EXPRESSION: VIRTUAL_FILENAMES_NUMMAT_EXP_PR,
     TAIGA_MUTATION: VIRTUAL_FILENAMES_NUMMAT_MUT_PR,
}

LATEST2FN_TABLE_PR = {
#    TAIGA_CN: VIRTUAL_FILENAMES_TABLE_CN_PR,
#     TAIGA_FUSION: VIRTUAL_FILENAMES_TABLE_FUSION_PR,
     TAIGA_MUTATION: VIRTUAL_FILENAMES_TABLE_MUT_PR,
}

In [ ]:
upload.makePRLvMatrices(virtual_ids=virtual_dryrun, files_nummat=LATEST2FN_NUMMAT_PR, files_table=LATEST2FN_TABLE_PR)

In [ ]:
upload.makeModelLvMatrices(virtual_ids=virtual_dryrun, files_nummat=LATEST2FN_NUMMAT_MODEL, files_table=LATEST2FN_TABLE_MODEL)

In [ ]:
upload.uploadAuxTables(taiga_ids=virtual_dryrun,)

In [ ]:
set(CCLE_expression.index) - set(OmicsExpressionProteinCodingGenesTPMLogp1.index)

In [ ]:
shared_cols = [c in CCLE_expression.columns if c in OmicsExpressionProteinCodingGenesTPMLogp1.columns]
CCLE_expression = CCLE_expression[shared_cols]

In [ ]:
CCLE_expression.corrwith(OmicsExpressionProteinCodingGenesTPMLogp1)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

OmicsSomaticMutations = tc.get(name='internal-22q2dryrun-5ffb', version=39, file='OmicsSomaticMutations')
OmicsSomaticMutations.columns

In [ ]:
OmicsSomaticMutations

In [ ]:
O_micsStructuralVariantsProfile = tc.get(name='dmc-22q2dryrun-0f8d', version=28, file='O​micsStructuralVariantsProfile')


In [ ]:
O_micsStructuralVariantsProfile

In [ ]:
OmicsLoH = tc.get(name='dmc-22q2dryrun-0f8d', file='OmicsLoH')
OmicsLoH

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

somaticMutations_profile = tc.get(name='mutations-latest-f263', version=8, file='somaticMutations_profile')

In [ ]:
somaticMutations_model = somaticMutations_profile[
            somaticMutations_profile["DepMap_ID"].isin(set(pr2model_dict.keys()))
        ].replace({"DepMap_ID": pr2model_dict})
somaticMutations_model.to_csv("somaticMutations_model.csv", index=False)

In [ ]:
tc.update_dataset(
    changes_description="add model level maf",
    dataset_permaname="mutations-latest-f263",
    upload_files=[            
        {
            "path": "somaticMutations_model.csv",
            "format": "TableCSV",
            "encoding": "utf-8",
        },
    ],
    upload_async=False,
    add_all_existing_files=True,
)

In [ ]:
d = upload.getPRToRelease()

In [ ]:
default_table = upload.makeDefaultModelTable(d['internal'])
pr2model_dict = dict(list(zip(default_table.ProfileID, default_table.ModelID)))

In [ ]:
pr2model_dict

In [ ]:
from depmap_omics_upload import tracker as track

mytracker = track.SampleTracker()

In [ ]:
pr_table = mytracker.read_pr_table()
seq_table = mytracker.add_model_cols_to_seqtable(cols=["ModelID"])

In [ ]:
import pandas as pd

for pr in pr_table.index:
    if (pr_table.loc[pr, "Datatype"] == "wgs" or pr_table.loc[pr, "Datatype"] == "wes") and not (pd.isna(pr_table.loc[pr, "MainSequencingID"])):
        cdsid = pr_table.loc[pr, "MainSequencingID"]
        if seq_table.loc[cdsid, "source"] == 'CCLE2':
            if seq_table.loc[cdsid, "ModelID"] not in OmicsCNGene.index:
                print(seq_table.loc[cdsid, "ModelID"])

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
OmicsCNGene = tc.get(name='internal-22q2dryrun-5ffb', version=86, file='OmicsCNSegmentsProfile')


In [ ]:
# grant gs access
import dalmatian as dm
ws = dm.WorkspaceManager("fccredits-silver-tan-7621/CCLE_v2").disable_hound()
ccle2samples = ws.get_samples()

In [ ]:
cols = ['hg38_rna_bam', 'hg38_rna_bai', 'hg38_wgs_bam', 'hg38_wgs_bai', 'hg38_wes_bam', 'hg38_wes_bai']

In [ ]:
from genepy.utils import helper as h
import pandas as pd

cmds = []
for i in ccle2samples.index:
    for c in ['hg38_wgs_bam', 'hg38_wgs_bai']:
        if pd.isna(ccle2samples.loc[i, c]) == False:
            if '/wgs/' in ccle2samples.loc[i, c] or '/wes/' in ccle2samples.loc[i, c]:
                print(ccle2samples.loc[i, c])
            cmds.append("gsutil -u broad-firecloud-ccle -m acl ch -R -g All:R " + ccle2samples.loc[i, c])

In [ ]:
h.parrun(cmds, cores=8)

In [ ]:
cmds

In [ ]:
old_maf = pd.read_csv("gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/submissions/cfeeb3ac-7eaa-48ac-8afa-8ba6316e9c93/run_vcf_to_depmap/ea5d21ec-b350-4ee2-a391-9d26e21ce96b/call-vcf_to_depmap/attempt-2/CDS-0Z9yIY-maf-coding_somatic-subset.csv.gz")

In [ ]:
old_maf

In [ ]:
new_maf = pd.read_csv("gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/58b92d2c-7d5e-483c-a7d9-ca3a952458d2/WGS_pipeline/5be4117d-bc64-4e51-aca4-c30dca3d170b/call-my_vcf_to_depmap/CDS-0Z9yIY-maf-coding_somatic-subset.csv.gz")